In [1]:
import os
import importlib
import time

import numpy as np
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import skimage
#from skimage import io as img_io
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
#from torchvision import transforms
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler

import load_data
import model

In [2]:
plt.style.use('Solarize_Light2')
%matplotlib inline
importlib.reload(load_data)
importlib.reload(model)

<module 'model' from '/home/uldo/work/kaggle/competitions/Humpback_Whale_Identification/code/model.py'>

In [3]:
BATCH_SIZE = 40

In [16]:
class ToTensor(object):
    
    """Convert ndarrays in sample to Tensors."""
    def __init__(self, image_size=128):
        self.image_size = image_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        
        """ The original code didn't expect gray scale images """
        
        gray_scale_image = torch.zeros(
            [self.image_size, self.image_size]
        ).shape == image.shape
        if gray_scale_image:
            image = np.stack((image,) * 3, axis=-1)
        #image = image.transpose((2, 0, 1))
        print("image.shape: ", image.shape)
        print("label: ", label)
        return {'image': torch.from_numpy(image), 'label': torch.tensor(label, dtype=torch.uint8)}

In [6]:
def prepare_loaders(dataset, valid_train_ratio=0.6):
    dataset_size = len(dataset)
    print("dataset_size: ", dataset_size)

    validation_subset_size = int(dataset_size * (1 - valid_train_ratio))
    print("validation_subset_size: ", validation_subset_size)

    indices = list(range(dataset_size))
    validation_indices = np.random.choice(indices, size=validation_subset_size, replace=False)
    train_indices = list(set(indices) - set(validation_indices))

    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)
    
    dataset_sizes = {
            'train': len(train_indices),
            'validation': len(validation_indices)
        }

    train_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=train_sampler)
    validation_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=validation_sampler)

    loaders = {
            'train': train_loader,
            'validation': validation_loader
        }

    return loaders, dataset_sizes

In [7]:
train_df = load_data.load_text_data('../input/train.csv')

In [17]:
dataset = load_data.HumpbackWhalesDataset(
    train_df,
    #transform=load_data.transforms.ToTensor()
    transform=ToTensor()
)

In [18]:
dataset_loaders, dataset_sizes = prepare_loaders(dataset)

dataset_size:  25361
validation_subset_size:  10144


In [19]:
train_loader = dataset_loaders['train']
validation_loader = dataset_loaders['validation']
train_dataset_size = dataset_sizes['train']
validation_dataset_size = dataset_sizes['validation']

In [20]:
print(train_dataset_size)
print(validation_dataset_size)

15217
10144


In [21]:
print(train_loader)

In [22]:
train_iter = iter(train_loader)

image.shape:  (700, 1050, 3)
label:  217
image.shape:  (620, 930, 3)
label:  86
image.shape:  (414, 1050, 3)
label:  448
image.shape:  (536, 1050, 3)
label:  4562
image.shape:  (525, 1050, 3)
label:  3
image.shape:  (600, 1050)
label:  3
image.shape:  (251, 740, 3)
label:  941
image.shape:  (700, 1050, 3)
label:  1255
image.shape:  (640, 1050, 3)
label:  37
image.shape:  (422, 1050, 3)
label:  2571
image.shape:  (250, 1050, 3)
label:  92
image.shape:  (450, 1050, 3)
label:  384
image.shape:  (450, 1050, 3)
label:  447
image.shape:  (175, 306, 3)
label:  3227
image.shape:  (525, 1050, 3)
label:  3
image.shape:  (342, 599, 3)
label:  768
image.shape:  (450, 1050, 3)
label:  3
image.shape:  (344, 802, 3)
label:  3512
image.shape:  (481, 1050, 3)
label:  3
image.shape:  (700, 1050, 3)
label:  210
image.shape:  (457, 811, 3)
label:  1827
image.shape:  (250, 1050, 3)
label:  3
image.shape:  (600, 1050, 3)
label:  737
image.shape:  (485, 848, 3)
label:  2694
image.shape:  (377, 1050, 3)
label

In [23]:
print(type(train_iter))

<class 'torch.utils.data.dataloader._DataLoaderIter'>


In [24]:
images, labels = train_iter.next()

RuntimeError: Traceback (most recent call last):
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 184, in default_collate
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 184, in <dictcomp>
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 164, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 700 and 620 in dimension 1 at /opt/conda/conda-bld/pytorch_1533672544752/work/at